In [16]:
from scipy.spatial import cKDTree
import pyfits, mechanize, bz2, os
from matplotlib import pyplot as pl
from numpy import *
from scipy.optimize import leastsq
import astropy.stats as sta
from scipy import ndimage
from glob import glob
import pandas as pd  #為了讀csv檔。csv檔是以逗點分割的檔案。
import requests
from bs4 import BeautifulSoup
from astropy.io import ascii
import concurrent.futures
from time import sleep
from astropy.table import Table
import numpy as np

In [2]:
cd /media/careyjohn/DATA/kepler_ptf_research/k2_c4

/media/careyjohn/DATA/kepler_ptf_research/k2_c4


In [3]:
df = pd.read_csv('ptf_ha_substraction_k2c4.csv')

In [26]:
epic = df['EPIC']
ha_ratio = df['fHA656/fHA663']
ha_sub = df['HA663-HA656']

In [27]:
ha_ratio_1 =[]
epic_1 = []
ha_sub_1 = []
for i,v in enumerate(ha_ratio):
    if ha_ratio[i] != 0. and ha_ratio[i] > 1.:
        ha_ratio_1.append(ha_ratio[i])
        epic_1.append(epic[i])
        ha_sub_1.append(ha_sub[i])

In [28]:
len(ha_ratio_1)

1626

In [17]:
np.savetxt("m-dwarfs_ptf_hasub-in-k2c4.txt", epic_1, fmt= '%i')

In [18]:
df2 = pd.read_csv('k2_c4_ptf_m-dwarfs.csv')

In [19]:
ra = df2['RA']
dec = df2['Dec']
epic_2 = df2['EPIC']

In [21]:
ra_1 = []
dec_1 = []
for i,v in enumerate(epic_1):
    for a,s in enumerate(epic_2):
        if epic_1[i] == epic_2[a]:
            ra_1.append(ra[a])
            dec_1.append(dec[a])

In [22]:
len(ra_1)

1626

In [25]:
pl.figure(figsize=(10,8))
pl.title('1626 M dwarfs have ptf hasub on Kepler K2 Campaign 4')
pl.plot(ra_1, dec_1, 'ro', markersize = 1.)
pl.xlim(49, 68)
pl.ylim(min(dec_1)-1, max(dec_1)+1)
pl.grid(True)
pl.xlabel('$RA  (deg)$', size = 15)
pl.ylabel('$Dec  (deg)$', size = 15)
pl.show()

In [36]:
os.chdir("/media/careyjohn/DATA/kepler_ptf_research/k2_c4/python_Kepler_data_k2c4/results")
column_names = ['EPIC','RA','Dec','HSV','fH656/fH663','Period']  

df = pd.DataFrame([ ], columns = column_names)
df.to_csv('K2_M_C4_epic_period.csv', index=False)

In [46]:
def append_csv(x1, x2, x3, x4, x5, x6):

    df = pd.read_csv('K2_M_C4_epic_period.csv')
    column_names = ['EPIC','RA','Dec','HSV','fH656/fH663','Period'] 
    
    if x1 not in list(df['EPIC'])  :
    
    
        df2 = pd.DataFrame([[x1, x2, x3, x4, x5, x6]], columns = column_names)

        df_add = df.append(df2)

        df_add.to_csv('K2_M_C4_epic_period.csv', index = False)
    #else:
        #print x3,'already exists'

In [44]:
from matplotlib import pyplot as plt
import pyfits, os, sys, matplotlib, pylab
import numpy as np
from gatspy import periodic
import pandas as pd

#os.chdir("C:\\Users\\CareyJohn\\kplr_data_test")  #change directory
#os.chdir("/media/careyjohn/OS/Users/CareyJohn/Desktop/PSROC_klpr/K2_LLC_FITS/k2_camp1/ktwo201323410_C1")

def k2_llc_analysis(epic, RA, Dec, HSV, flux_r):
    
    os.chdir("/media/careyjohn/DATA/kepler_ptf_research/k2_c4/python_Kepler_data_k2c4/results/")
    
    df =  pd.read_csv('K2_M_C4_epic_period.csv')
    
    if epic not in list(df['EPIC']):
    
        ID = "ktwo"+str(epic)

        #os.chdir("C:/Users/CareyJohn/Desktop/PSROC_klpr/K2_LLC_FITS/k2_camp1/"+ID+"_C1")
        #os.chdir("D:/kepler_ptf_research/k2_c1/python_Kepler_data_k2c1"+ID+"_C1")
        
        #os.chdir("D:/kepler_ptf_research/k2_c4/python_Kepler_data_k2c4")
        os.chdir("/media/careyjohn/DATA/kepler_ptf_research/k2_c4/python_Kepler_data_k2c4")
        #nowpath = os.getcwd()
        #print (nowpath)

        data = pyfits.open(ID+"-c04_llc.fits")  #open the kepler fits data
        print (data.info())

        time = data[1].data.field('TIME')
        sap_flux=data[1].data.field('SAP_FLUX')
        sap_flux_err=data[1].data.field('SAP_FLUX_ERR')
        pdcsap_flux=data[1].data.field('PDCSAP_FLUX')
        pdcsap_flux_err=data[1].data.field('PDCSAP_FLUX_ERR')


        #plt.plot(time,pdcsap_flux)
        #plt.show()

        lc=zip(time,sap_flux,sap_flux_err,pdcsap_flux,pdcsap_flux_err)  #zip the open data 

        #print(list(lc))
        #os.chdir("..")
        #os.chdir("D:/kepler_ptf_research/k2_c1/python_Kepler_data_k2c4/results/")
        os.chdir("/media/careyjohn/DATA/kepler_ptf_research/k2_c4/python_Kepler_data_k2c4/results")
        if not os.path.exists("ktwo"+str(epic)+"_C4"):
            os.mkdir("ktwo"+str(epic)+"_C4")
        
        #os.chdir("D:/kepler_ptf_research/k2_c1/python_Kepler_data_k2c4/results/ktwo"+str(epic)+"_C1")
        os.chdir("/media/careyjohn/DATA/kepler_ptf_research/k2_c4/python_Kepler_data_k2c4/results/ktwo"+str(epic)+"_C4")
        if not os.path.exists(ID+"-c04_llc.fits.lc"):

            output_file = ID+"-c04_llc.fits.lc"  #write the zip of the open data in a text book
            out = open(output_file,'w')
            for i in lc:
                write = '%s %s %s %s %s' %(i[0],i[1],i[2],i[3],i[4])
                out.write(write+'\n')
            out.close()

        filej = open( ID+"-c04_llc.fits.lc")

        datar = np.loadtxt( filej, dtype={'names':('time', '2', '3', 'flux', 'err'), 'formats':('f4','f4','f4','f4','f4')})



        masknan = -np.isnan(datar['flux']) # 遮閉掉nan的值, 加上'-'號是為了讓nan=false;數值=true
        masknan_err = -np.isnan(datar['err'])

        #print(data['flux'])
        #print(np.isnan(data['flux']))
        #print(masknan)
        #print(data['flux'][masknan])

        time = datar['time'][masknan]
        flux = datar['flux'][masknan]/np.mean(datar['flux'][masknan])  # 之後會被動手腳
        flux_raw = datar['flux'][masknan]/np.mean(datar['flux'][masknan])# 為了上一個而留下的替代品
        err = datar['err'][masknan_err]/np.mean(datar['err'][masknan])
        #print(time)
        #print(flux)
        #print(flux_raw)

        fluxC = np.zeros(len(flux))
        errC = np.zeros(len(err))

        separationtime=[]
        separationtime.append(0)

        for i in range(len(time)-1):
            if time[i+1]-time[i] > 3*(time[2]-time[1]):
                separationtime.append(i)
        separationtime.append(len(time))
        binsize = 10
        for k in range(len(separationtime)-1):
            #print(k)
            #take 10 points mean and replace it 
            for i in range(separationtime[k],separationtime[k+1]):

                if i >= (separationtime[k]+binsize) and i <= (separationtime[k+1]-binsize):
                    fluxC[i] = np.median(flux[i-binsize:i+binsize])
                    errC[i] = np.median(flux[i-binsize:i+binsize])
                else:
                    fluxC[i] = flux[i]
                    errC[i] = err[i]
            #plt.plot(time,flux_raw)
            #plt.scatter(time,flux_raw)
            #plt.plot(time,fluxC)
            #plt.scatter(time,fluxC)
            #plt.show()
            #plt.clf()
            #print (fluxC)
            #print (flux_raw)

            #obseverd subtract calculate
            fluxdiff = flux - fluxC

            #remove the peak value which > 1 sigam
            maskpeak = fluxdiff > 1.0*fluxdiff.std()

            for n,i in enumerate(maskpeak):
                if i:
                    if n >= (separationtime[k]+binsize) and n <= (separationtime[k+1]-binsize):
                        flux[n] = np.median(flux[n-binsize:n+binsize])
                    else:
                        flux[n] = flux[n]
        #plt.plot(time,flux_raw, 'r')
        #plt.plot(time,flux_raw, '.')
        #plt.scatter(time,flux_raw)
        #plt.plot(time,fluxC)
        #plt.scatter(time,fluxC)
        #plt.xlabel('Time(BJD - 2454833)')
        #plt.ylabel('Normalized flux')
        #plt.xlim(1976,1980)
        #plt.show()
        #plt.clf()
        #print (fluxC)
        #print (flux_raw)

        model = periodic.LombScargle(fit_period = True)

        model.optimizer.period_range = ( 0.1, 70.0)

        model.fit(time, flux);

        model.best_period

        P = model.best_period

        print(epic, P)
        
        os.chdir("/media/careyjohn/DATA/kepler_ptf_research/k2_c4/python_Kepler_data_k2c4/results")
        
        if P > 70.0:   #這裡LSP的週期測試上限為70.0 days，若分析時超過79.0，則代表分析資料或過程有問題，不能寫入csv檔並儲存
            append_csv(epic, RA, Dec, HSV, flux_r, 0)  
        else:
            append_csv(epic, RA, Dec, HSV, flux_r, P)
            
        os.chdir("/media/careyjohn/DATA/kepler_ptf_research/k2_c4/python_Kepler_data_k2c4")
        #os.chdir("D:/kepler_ptf_research/k2_c1/python_Kepler_data_k2c1")

In [47]:
for i,v in enumerate(epic_1):
    k2_llc_analysis(int(epic_1[i]), ra_1[i], dec_1[i], ha_sub_1[i], ha_ratio_1[i])

Filename: ktwo210310510-c04_llc.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      54   ()              
1    LIGHTCURVE  BinTableHDU    155   3470R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
2    APERTURE    ImageHDU        48   (13, 13)     int32   
None


/home/careyjohn/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:67: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.
/home/careyjohn/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:68: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


Finding optimal frequency:
 - Estimated peak width = 0.0886
 - Using 5 steps per peak; omega_step = 0.0177
 - User-specified period range:  0.1 to 70
 - Computing periods at 3540 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 1000 steps
(210310510, 26.461385927802947)
Filename: ktwo210312956-c04_llc.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      54   ()              
1    LIGHTCURVE  BinTableHDU    155   3470R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
2    APERTURE    ImageHDU        48   (12, 11)     int32   
None
Finding optimal frequency:
 - Estimated peak width = 0.0886
 - Using 5 steps per peak; omega_step = 0.0177
 - User-specified period range:  0.1 to 70
 - Computing periods at 3540 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 1000 steps
(210312956, 32.594419647611524)
Filename: ktwo210314232-c04_llc.fits
No.    Name         Type      Cards   Dimensions   Format
0   